In [ ]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

load_dotenv()

model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# 여러 PDF 처리
files = [
    "특허_실용신안 심사기준.pdf"
]

all_docs = []
for file in files:
    loader = PyPDFLoader(file)
    docs = loader.load()
    all_docs.extend(docs)

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(all_docs)


In [ ]:
import pdfplumber
import os
from langchain.schema import Document

# PDF에서 텍스트 추출
docs = []

# 폴더 내 파일 가져오기
path = 'pdf_files/'
file_list = os.listdir(path)

for doc_num, file_name in enumerate(file_list):
    file_path = os.path.join(path, file_name)
    try:
        extracted_text = ""  # PDF 한 파일의 전체 텍스트를 저장할 변수
        with pdfplumber.open(file_path) as pdf_file:
            for i, page in enumerate(pdf_file.pages):
                try:
                    # 텍스트 추출
                    text = page.extract_text()
                    if text:
                        extracted_text += text + "\n"  # 각 페이지의 텍스트를 합침
                except Exception as e:
                    print(f"페이지 {i + 1}에서 오류 발생: {e}")

        # 한 PDF 파일의 전체 텍스트를 하나의 Document로 저장
        if extracted_text.strip():  # 추출된 텍스트가 있으면 저장
            document = Document(
                page_content=extracted_text,
                metadata={"doc_number": doc_num + 1, "file_name": file_name}
            )
            docs.append(document)
        else:
            print(f"파일 '{file_name}'에서 텍스트 추출이 없습니다.")

    except Exception as e:
        print(f"파일 '{file_name}'에서 오류 발생: {e}")

# 텍스트 추출 결과 확인
print(f"총 {len(docs)}개의 PDF 파일에서 텍스트를 추출했습니다.")

In [ ]:
# 이미지로 처리되어 내용이 없는 파일 삭제
print(docs[0].page_content)

docs = docs[1:]

In [ ]:
import json

# 과실 비율 데이터 읽어오기
# 폴더 내에 있는 파일 모두 가져오기
with open('patent_cases_all_pages.json', 'r', encoding='utf-8') as f:
    file = json.load(f)


# JSON 데이터를 Document로 변환
def nested_json_to_documents(json_data):
    docs = []  # 문서 리스트 초기화
    # 중첩된 리스트를 순회하며 평탄화
    for entry in json_data:  # 최상위 리스트 순회
        content = (
            f"사건 제목: {entry['제목']}\n"
            f"상세 내용 확인 링크: {entry['링크']}\n"
        )
        docs.append(Document(page_content=content))  # Document 객체 추가
    return docs

docs_rate = nested_json_to_documents(file)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from tqdm import tqdm
import json
import os

# 문서 불러오기
with open("patent_cases_all_pages.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

docs = []
for entry in json_data:
    title = entry.get("제목", "")
    link = entry.get("링크", "")
    content = f"제목: {title}\n링크: {link}"
    metadata = {k: v for k, v in entry.items()}
    docs.append(Document(page_content=content, metadata=metadata))

# GPT 모델 설정
model = ChatOpenAI(model_name="gpt-4o")

# 프롬프트 설정
summary_prompt = ChatPromptTemplate.from_messages([
    ('system', '주어진 문서의 "제목"과 "링크"만 보고 사건의 성격을 한 문장으로 요약해줘. 마치 뉴스 기사처럼 요약해줘. 예: "이 사건은 A와 B 간의 어떠한 특허권 분쟁으로, 어떤 법적 책임 여부가 쟁점이 되었으며 판결은 어떻게 되었다"'),
    ('user', '{content}')
])

# 요약 함수
def summarize_accident(accident_text):
    try:
        messages = summary_prompt.format_messages(content=accident_text)
        response = model(messages)
        return response.content
    except Exception as e:
        print(f"요약 중 오류 발생: {e}")
        return ""

# 요약 문서 생성 함수
def summary_docs(original_doc, summary_text):
    title = original_doc.metadata.get("제목", "")
    link = original_doc.metadata.get("링크", "")
    combined_text = f"제목: {title}\n링크: {link}\n요약: {summary_text}"
    return Document(
        page_content=combined_text,
        metadata={**original_doc.metadata, 'summary': summary_text}
    )

# 캐시 파일 경로
CACHE_PATH = "summaries_cache.json"

# 캐시 불러오기
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        cached_summaries = json.load(f)
else:
    cached_summaries = {}

# 요약 실행
summarized_docs = []
total = len(docs)

for i, doc in enumerate(tqdm(docs, desc="요약 중", unit="문서", ncols=100), 1):
    title = doc.metadata.get("제목", "")
    print(f"[{i}/{total}] 처리 중: {title}")

    if title in cached_summaries:
        summary = cached_summaries[title]
    else:
        summary = summarize_accident(doc.page_content)
        cached_summaries[title] = summary

    summarized_docs.append(summary_docs(doc, summary))

# 캐시 저장
with open(CACHE_PATH, "w", encoding="utf-8") as f:
    json.dump(cached_summaries, f, ensure_ascii=False, indent=2)

print(f"\n요약 완료 ✅ 총 문서 수: {len(summarized_docs)}")


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from uuid import uuid4
from typing import List
import json

# 1. OpenAI 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. Document 로딩 (예: 요약문 기준)
with open("summaries_cache.json", "r", encoding="utf-8") as f:
    summary_data = json.load(f)

summarized_docs = [
    Document(page_content=summary, metadata={"title": title})
    for title, summary in summary_data.items()
]

# 3. UUID 생성
uuids1 = [f"docs_{i+1}" for i in range(len(summarized_docs))]

# 4. FAISS 벡터 저장 함수
def build_vectorstore_batched(docs: List[Document], ids: List[str], batch_size=100) -> FAISS:
    stores = []
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        store = FAISS.from_documents(documents=batch_docs, ids=batch_ids, embedding=embeddings)
        stores.append(store)

    base_store = stores[0]
    for store in stores[1:]:
        base_store.merge_from(store)
    return base_store

# 5. 벡터 생성 및 저장
vector_store_law = build_vectorstore_batched(summarized_docs, uuids1)
vector_store_law.save_local("vector_store_law", index_name="index")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from typing import List
import json

# 1. OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. FAISS 벡터 저장 함수 정의
def build_vectorstore_batched(docs: List[Document], ids: List[str], batch_size=100) -> FAISS:
    if not docs:
        raise ValueError("❌ 입력된 문서 리스트가 비어 있습니다.")
    stores = []
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        store = FAISS.from_documents(documents=batch_docs, ids=batch_ids, embedding=embeddings)
        stores.append(store)
    base_store = stores[0]
    for store in stores[1:]:
        base_store.merge_from(store)
    return base_store

# 3. summaries_cache.json 로딩
with open("summaries_cache.json", "r", encoding="utf-8") as f:
    summary_data = json.load(f)

# 4. 문서 분리
docs_law = []
docs_situation = []
docs_rate = []

for title, summary in summary_data.items():
    doc = Document(page_content=summary, metadata={"title": title})
    if "침해" in summary or "등록무효" in summary:
        docs_law.append(doc)
    elif "사실관계" in summary or "상황" in summary:
        docs_situation.append(doc)
    elif "보상" in summary or "배상" in summary:
        docs_rate.append(doc)

# 5. UUID 생성
uuids_law = [f"law_{i+1}" for i in range(len(docs_law))]
uuids_situation = [f"situation_{i+1}" for i in range(len(docs_situation))]
uuids_rate = [f"rate_{i+1}" for i in range(len(docs_rate))]

# 6. 벡터 저장
if docs_law:
    vector_store_law = build_vectorstore_batched(docs_law, uuids_law)
    vector_store_law.save_local("vector_store_law", index_name="index")
    print("✅ vector_store_law 저장 완료")

if docs_situation:
    vector_store_situation = build_vectorstore_batched(docs_situation, uuids_situation)
    vector_store_situation.save_local("vector_store_situation", index_name="index")
    print("✅ vector_store_situation 저장 완료")

if docs_rate:
    vector_store_rate = build_vectorstore_batched(docs_rate, uuids_rate)
    vector_store_rate.save_local("vector_store_rate", index_name="index")
    print("✅ vector_store_rate 저장 완료")


In [ ]:
#불러오기
vector_store_law = FAISS.load_local(
    "vector_store_law",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
    index_name="index"
)

In [ ]:
#저장 잘 되었는지 확인용
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1. 임베딩 모델 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. 벡터 스토어 불러오기 (침해 관련 사건 벡터 저장소)
vector_store_law = FAISS.load_local(
    "vector_store_law",              # 저장된 폴더 경로
    embeddings=embedding_model,
    index_name="index",              # 저장 당시 사용한 index_name
    allow_dangerous_deserialization=True
)

# 3. 검색 쿼리 작성
query = "상표권 침해와 관련된 판례 알려줘"

# 4. 유사도 검색 실행
results = vector_store_law.similarity_search(query, k=3)

# 5. 결과 출력
for i, res in enumerate(results, 1):
    title = res.metadata.get("title", "[제목 없음]")
    print(f"\n🔹 결과 {i}: {title}")
    print(res.page_content)



🔹 결과 1: [상표]솔표 사건(특허법원 2023허29)
이 사건은 솔표 상표를 둘러싼 분쟁으로, 상표권 침해 여부가 쟁점이 되었으며 특허법원에서 판결이 이루어졌다.

🔹 결과 2: [상표]조선협객전 사건(특허법원 2023허12916)
이 사건은 "조선협객전"이라는 상표와 관련된 분쟁으로, 상표권 침해 여부가 쟁점이 되었으며 특허법원에서 판결이 이루어졌다.

🔹 결과 3: [민사]소위 명품 리폼 영업을 하면서 원고의 상표를 그대로 표시한 사건에서, 피고는 상표권침해금지 및 손해배상 의무가 있다고 판단한 예(특허법원 2023나11283)
이 사건은 명품 리폼 영업을 하면서 원고의 상표를 무단으로 사용한 피고가 상표권 침해 및 손해배상 책임이 있는지에 대한 분쟁으로, 법원은 피고에게 상표권침해금지 및 손해배상 의무가 있다고 판결하였다.
